# Lesson 8: The Core of the RISC Zero STARK

In this lesson, we'll combine concepts from previous lessons to understand the core part of the STARK protocol. We'll examine how all components work together to create a proof.

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Dict
import matplotlib.pyplot as plt

@dataclass
class STARKProof:
    """Structure for STARK proof."""
    trace_commitments: Dict[str, bytes]  # Commitments for each trace column
    constraint_evaluation: List[float]   # Constraint evaluations
    fri_layers: List[np.ndarray]        # FRI protocol layers
    merkle_proofs: List[List[bytes]]    # Merkle proofs

class STARKProver:
    """Simplified version of STARK prover."""
    
    def __init__(self, trace, domain):
        self.trace = trace
        self.domain = domain
        
    def create_trace_commitments(self):
        """Creates commitments for the trace."""
        from lesson_5_zk_commitments import create_merkle_tree
        
        commitments = {}
        for column in self.trace.columns[:3]:  # Only registers
            values = self.trace[column].values
            merkle_root = create_merkle_tree(values)
            commitments[column] = merkle_root.hash
        return commitments
    
    def evaluate_constraints(self):
        """Evaluates constraint values."""
        from lesson_6_constraint_polynomials import create_constraint_polynomial
        from lesson_7_mixing_constraints import create_mixed_constraint
        
        constraints = []
        for i in range(len(self.domain)):
            values = [self.trace[f'Register {j+1}'].values[i] for j in range(3)]
            constraint = create_constraint_polynomial(*values)
            constraints.append(constraint)
            
        # Mix constraints
        mixing_coeffs = np.random.rand(len(constraints))
        mixed = create_mixed_constraint(constraints, mixing_coeffs)
        
        return [float(mixed.eval(x)) for x in self.domain]
    
    def create_fri_layers(self, polynomial_values, num_layers=3):
        """Creates FRI protocol layers."""
        layers = [np.array(polynomial_values)]
        
        for _ in range(num_layers):
            prev_layer = layers[-1]
            next_layer = []
            
            for i in range(0, len(prev_layer), 2):
                if i + 1 < len(prev_layer):
                    combined = (prev_layer[i] + prev_layer[i + 1]) / 2
                else:
                    combined = prev_layer[i]
                next_layer.append(combined)
                
            layers.append(np.array(next_layer))
            
        return layers
    
    def generate_proof(self) -> STARKProof:
        """Generates complete STARK proof."""
        # 1. Create trace commitments
        commitments = self.create_trace_commitments()
        
        # 2. Evaluate constraints
        constraint_values = self.evaluate_constraints()
        
        # 3. Create FRI layers
        fri_layers = self.create_fri_layers(constraint_values)
        
        # 4. Create Merkle proofs for each layer
        merkle_proofs = [[b'proof'] * len(layer) for layer in fri_layers]  # Simplified
        
        return STARKProof(
            trace_commitments=commitments,
            constraint_evaluation=constraint_values,
            fri_layers=fri_layers,
            merkle_proofs=merkle_proofs
        )

## Demonstrating Proof Generation

In [ ]:
# Import necessary functions
from lesson_1_execution_trace import create_fibonacci_trace
from lesson_3_padding_trace import pad_trace_to_power_of_two

# Create trace and domain
trace = create_fibonacci_trace(3, 5)
padded_trace = pad_trace_to_power_of_two(trace)
domain = np.linspace(-1, 1, len(padded_trace))

# Create prover and generate proof
prover = STARKProver(padded_trace, domain)
proof = prover.generate_proof()

# Visualize proof components
plt.figure(figsize=(15, 10))

# Plot constraint values
plt.subplot(2, 1, 1)
plt.plot(domain, proof.constraint_evaluation, 'b-', label='Constraint Values')
plt.title('Constraint Evaluation')
plt.xlabel('Domain')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

# Plot FRI layers
plt.subplot(2, 1, 2)
for i, layer in enumerate(proof.fri_layers):
    x = np.linspace(-1, 1, len(layer))
    plt.plot(x, layer, label=f'FRI Layer {i+1}')

plt.title('FRI Protocol Layers')
plt.xlabel('Domain')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Print proof information
print("\nProof Information:")
print(f"Number of FRI layers: {len(proof.fri_layers)}")
print("\nFRI layer sizes:")
for i, layer in enumerate(proof.fri_layers):
    print(f"Layer {i+1}: {len(layer)} elements")

print("\nTrace column commitments:")
for column, commitment in proof.trace_commitments.items():
    print(f"{column}: {commitment.hex()[:16]}...")

## Key Components of STARK Protocol

1. **Proof Structure**:
   - Execution trace commitments
   - Polynomial constraint evaluations
   - FRI protocol layers
   - Merkle proofs for verification

2. **Proof Generation Steps**:
   - Create and commit to execution trace
   - Transform rules into polynomial constraints
   - Mix constraints
   - Apply FRI protocol
   - Create Merkle proofs

3. **Proof Properties**:
   - Transparency: all steps can be verified
   - Scalability: logarithmic proof size
   - Zero-knowledge: doesn't reveal execution details

In the next lesson, we'll examine the DEEP technique that improves protocol efficiency.